In [ ]:
"""
This notebook builds a small Monte Carlo programm to create data of the exact cover problem.
"""

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [71]:
class MC_EC3:
    # This is a class for the exact coer 3 problem. 
    
    def __init__(self, num_spins):
        # Construct an instance with num_spins of desired spins.
        self.num_spins = num_spins
    
    def random_clauses(self, num_clauses):
        # Sample num_clauses random clauses.
        self.num_clauses = num_clauses
        clauses = np.zeros((self.num_spins, 3), dtype=int)
        for i in range(self.num_spins):
            # Sample three spin indices.
            clauses[i] = np.random.choice(np.arange(self.num_spins), 3, replace=False)
        self.clauses = clauses
        
    def initial_state(self):
        # States contain 1, -1 spin orientations.
        self.state = (-1)**np.random.randint(0, 2, self.num_spins)
        
    def hamiltonian(self):
        # Construct classical Hamiltonian from given state and clauses.
        energy = 0
        for i in range(self.num_clauses):
            energy += (self.state[self.clauses[i]].sum() - 1)**2
        self.energy = energy
        return energy
    
    def flip_spin(self, i):
        self.state[i] = - self.state[i]
        

In [72]:
mc_ec3 = MC_EC3(7)

In [73]:
mc_ec3.random_clauses(6)

In [75]:
mc_ec3.initial_state()

In [80]:
mc_ec3.hamiltonian()

4

In [79]:
mc_ec3.state

array([-1,  1,  1,  1, -1, -1,  1])

In [78]:
mc_ec3.flip_spin(0)